In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
sns.set(style="ticks")
plt.rc('figure', figsize=(6, 3.7), dpi=100)
plt.rc('axes', labelpad=20, facecolor="#ffffff", 
       linewidth=0.4, grid=True, labelsize=10)
plt.rc('patch', linewidth=0)
plt.rc('xtick.major', width=0.2)
plt.rc('ytick.major', width=0.2)
plt.rc('grid', color='#EEEEEE', linewidth=0.25)
plt.rc('font', family='Arial', weight='400', size=10)
plt.rc('text', color='#282828')
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('savefig', pad_inches=0.3, dpi=300)

## Reading the Dataset

The first step is reading the dataset from the csv file we downloaded. We will use the `read_csv()` function from `Pandas` Python package:

In [ ]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("../input/AmesHousing.csv")

## Getting A Feel of the Dataset

Let's display the first few rows of the dataset to get a feel of it:

In [ ]:
# Configuring float numbers format
pd.options.display.float_format = '{:20.2f}'.format
dataset.head(n=5)

In [ ]:
dataset.describe(include=[np.number], percentiles=[.5]) \
    .transpose().drop("count", axis=1)

In [ ]:
dataset.describe(include=[np.object]).transpose() \
    .drop("count", axis=1)


### Solving The Missing Values

We should deal with the problem of missing values because some machine learning models don't accept data with missing values. Firstly, let's see the number of missing values in our dataset. We want to see the number and the percentage of missing values for each column that actually contains missing values.

In [ ]:
# Getting the number of missing values in each column
num_missing = dataset.isna().sum()
# Excluding columns that contains 0 missing values
num_missing = num_missing[num_missing > 0]
# Getting the percentages of missing values
percent_missing = num_missing * 100 / dataset.shape[0]
# Concatenating the number and perecentage of missing values 
# into one dataframe and sorting it
pd.concat([num_missing, percent_missing], axis=1, 
          keys=['Missing Values', 'Percentage']).\
          sort_values(by="Missing Values", ascending=False)

Now we start dealing with these missing values.

##### Pool QC

The percentage of missing values in `Pool QC` column is 99.56% which is very high. We think that a missing value in this column denotes that the corresponding house doesn't have a pool. To verify this, let's take a look at the values of `Pool Area` column:

In [ ]:
dataset["Pool Area"].value_counts()

In [ ]:
dataset["Pool QC"].fillna("No Pool", inplace=True)

##### Misc Feature

The percentage of missing values in Pool QC column is 96.38% which is very high also. Let's take a look at the values of `Misc Val` column:

In [ ]:
dataset["Misc Val"].value_counts()

We can see that `Misc Val` column has 2827 entries with a value of 0. `Misc Feature` has 2824 missing values. Then, as with `Pool QC`, we can say that each house without a "miscellaneous feature" has a missing value in `Misc Feature` column and a value of 0 in `Misc Val` column. So let's fill the missing values in `Misc Feature` column with `"No Feature"`:

In [ ]:
dataset['Misc Feature'].fillna('No feature', inplace=True)

##### Alley,  Fence, and Fireplace Qu

According to the dataset documentation, `NA` in `Alley`, `Fence`, and `Fireplace Qu` columns denotes that the house doesn't have an alley, fence, or fireplace. So we fill in the missing values in these columns with `"No Alley"`, `"No Fence"`, and `"No Fireplace"` accordingly:

In [ ]:
dataset['Alley'].fillna('No Alley', inplace=True)
dataset['Fence'].fillna('No Fence', inplace=True)
dataset['Fireplace Qu'].fillna('No Fireplace', inplace=True)

##### Lot Frontage

As we saw previously, `Lot Frontage` represents the linear feet of street connected to the house. So we assume that the missing values in this column indicates that the house is not connected to any street, and we fill in the missing values with 0:

In [ ]:
dataset['Lot Frontage'].fillna(0, inplace=True)

##### Garage Cond, Garage Qual, Garage Finish, Garage Yr Blt, Garage Type, Garage Cars, and Garage Area

According to the dataset documentation, `NA` in `Garage Cond`, `Garage Qual`, `Garage Finish`, and `Garage Type` indicates that there is no garage in the house. So we fill in the missing values in these columns with `"No Garage"`. We notice that `Garage Cond`, `Garage Qual`, `Garage Finish`, `Garage Yr Blt` columns have 159 missing values, but `Garage Type` has 157 and both `Garage Cars` and `Garage Area` have one missing value. Let's take a look at the row that contains the missing value in `Garage Cars`:

In [ ]:
garage_columns = [col for col in dataset.columns if col.startswith("Garage")]
dataset[dataset['Garage Cars'].isna()][garage_columns]

We can see that this is the same row that contains the missing value in `Garage Area`, and that all garage columns except `Garage Type` are null in this row, so we will fill the missing values in `Garage Cars` and `Garage Area` with 0.

We saw that there are 2 rows where `Garage Type` is not null while `Garage Cond`, `Garage Qual`, `Garage Finish`, and `Garage Yr Blt` columns are null. Let's take a look at these two rows:

In [ ]:
dataset[~pd.isna(dataset['Garage Type']) & 
        pd.isna(dataset['Garage Qual'])][garage_columns]

We will replace the values of `Garage Type` with `"No Garage"` in these two rows also.

For `Garage Yr Blt`, we will fill in missing values with 0 since this is a numerical column:

In [ ]:
dataset['Garage Cars'].fillna(0, inplace=True)
dataset['Garage Area'].fillna(0, inplace=True)

dataset.loc[~pd.isna(dataset['Garage Type']) & 
            pd.isna(dataset['Garage Qual']), "Garage Type"] = "No Garage"

for col in ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']:
    dataset[col].fillna('No Garage', inplace=True)
    
dataset['Garage Yr Blt'].fillna(0, inplace=True)

##### Bsmt Exposure, BsmtFin Type 2, BsmtFin Type 1, Bsmt Qual, Bsmt Cond, Bsmt Half Bath, Bsmt Full Bath, Total Bsmt SF, Bsmt Unf SF, BsmtFin SF 2, and BsmtFin SF 1


According to the dataset documentation, `NA` in any of the first five of these columns indicates that there is no basement in the house. So we fill in the missing values in these columns with `"No Basement"`. We notice that the first five of these columns have 80 missing values, but `BsmtFin Type 2` has 81, `Bsmt Exposure` has 83, `Bsmt Half Bath` and `Bsmt Full Bath` each has 2, and each of the others has 1. Let's take a look at the rows where `Bsmt Half Bath` is null:

In [ ]:
bsmt_columns = [col for col in dataset.columns if "Bsmt" in col]
dataset[dataset['Bsmt Half Bath'].isna()][bsmt_columns]

We can see that these are the same rows that contain the missing values in `Bsmt Full Bath`, and that one of these two rows is contains the missing value in each of `Total Bsmt SF`, `Bsmt Unf SF`, `BsmtFin SF 2`, and `BsmtFin SF 1` columns. We notice also that `Bsmt Exposure`, `BsmtFin Type 2`, `BsmtFin Type 1`, `Bsmt Qual`, and `Bsmt Cond` are null in these rows, so we will fill the missing values in `Bsmt Half Bath`, `Bsmt Full Bath`, `Total Bsmt SF`, `Bsmt Unf SF`, `BsmtFin SF 2`, and `BsmtFin SF 1` columns with 0.

We saw that there are 3 rows where `Bsmt Exposure` is null while `BsmtFin Type 1`, `Bsmt Qual`, and `Bsmt Cond` are not null. Let's take a look at these three rows:

In [ ]:
dataset[~pd.isna(dataset['Bsmt Cond']) & 
        pd.isna(dataset['Bsmt Exposure'])][bsmt_columns]

We will fill in the missing values in `Bsmt Exposure` for these three rows with `"No"`. According to the dataset documentation, `"No"` for `Bsmt Exposure` means "No Exposure":

Let's now take a look at the row where `BsmtFin Type 2` is null while `BsmtFin Type 1`, `Bsmt Qual`, and `Bsmt Cond` are not null:

In [ ]:
dataset[~pd.isna(dataset['Bsmt Cond']) & 
        pd.isna(dataset['BsmtFin Type 2'])][bsmt_columns]

We will fill in the missing value in `BsmtFin Type 2` for this row with `"Unf"`. According to the dataset documentation, `"Unf"` for `BsmtFin Type 2` means "Unfinished":

In [ ]:
for col in ["Bsmt Half Bath", "Bsmt Full Bath", "Total Bsmt SF", 
            "Bsmt Unf SF", "BsmtFin SF 2", "BsmtFin SF 1"]:
    dataset[col].fillna(0, inplace=True)

dataset.loc[~pd.isna(dataset['Bsmt Cond']) & 
            pd.isna(dataset['Bsmt Exposure']), "Bsmt Exposure"] = "No"
dataset.loc[~pd.isna(dataset['Bsmt Cond']) & 
            pd.isna(dataset['BsmtFin Type 2']), "BsmtFin Type 2"] = "Unf"

for col in ["Bsmt Exposure", "BsmtFin Type 2", 
            "BsmtFin Type 1", "Bsmt Qual", "Bsmt Cond"]:
    dataset[col].fillna("No Basement", inplace=True)

##### Mas Vnr Area and Mas Vnr Type

Each of these two columns have 23 missing values. We will fill in these missing values with `"None"` for `Mas Vnr Type` and with 0 for `Mas Vnr Area`. We use `"None"` for `Mas Vnr Type` because in the dataset documentation, `"None"` for `Mas Vnr Type` means "None" (i.e. no masonry veneer):

In [ ]:
dataset['Mas Vnr Area'].fillna(0, inplace=True)
dataset['Mas Vnr Type'].fillna("None", inplace=True)

##### Electrical

This column has one missing value. We will fill in this value with the mode of this column:

In [ ]:
dataset['Electrical'].fillna(dataset['Electrical'].mode()[0], inplace=True)

Now let's check if there is any remaining missing value in our dataset:

In [ ]:
dataset.isna().values.sum()

This means that our dataset is now complete; it doesn't contain any missing value anymore.

## Outlier Removal

In the paper in which our dataset was introduced by De Cock (2011), the author states that there are five unusual values and outliers in the dataset, and encourages the removal of these outliars. He suggested plotting `SalePrice` against `Gr Liv Area` to spot the outliers. We will do that now:

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

plt.scatter(x=dataset['Gr Liv Area'], y=dataset['SalePrice'], 
            color="orange", edgecolors="#000000", linewidths=0.5);
plt.xlabel("Gr Liv Area"); plt.ylabel("SalePrice");

We can clearly see the five values meant by the authour in the plot above. Now, we will remove them from our dataset. We can do so by keeping data points that have `Gr Liv Area` less than 4,000. But first we take a look at the dataset rows that correspond to these unusual values:

In [ ]:
outlirt_columns = ["Gr Liv Area"] + \
                  [col for col in dataset.columns if "Sale" in col]
dataset[dataset["Gr Liv Area"] > 4000][outlirt_columns]

Now we remove them:

In [ ]:
dataset = dataset[dataset["Gr Liv Area"] < 4000]

In [ ]:
plt.scatter(x=dataset['Gr Liv Area'], y=dataset['SalePrice'], 
            color="orange", edgecolors="#000000", linewidths=0.5);
plt.xlabel("Gr Liv Area"); plt.ylabel("SalePrice");

To avoid problems in modeling later, we will reset our dataset index after removing the outlier rows, so no gaps remain in our dataset index:

In [ ]:
dataset.reset_index(drop=True, inplace=True)

## Deleting Some Unimportant Columns

We will delete columns that are not useful in our analysis. The columns to be deleted are `Order` and `PID`:

In [ ]:
dataset.drop(['Order', 'PID'], axis=1, inplace=True)

<h1 id="eda">Exploratory Data Analysis</h1>

In this section, we will explore the data using visualizations. This will allow us to understand the data and the relationships between variables better, which will help us build a better model.

## Target Variable Distribution

Our dataset contains a lot of variables, but the most important one for us to explore is the target variable. We need to understand its distribution. First, we start by plotting the violin plot for the target variable. 

In [ ]:
sns.violinplot(x=dataset['SalePrice'], inner="quartile", color="#36B37E");

We can see from the plot that most house prices fall between 100,000 and 250,000. The dashed lines represent the locations of the three quartiles Q1, Q2 (the median), and Q3. Now let's see the box plot of `SalePrice`:

In [ ]:
sns.boxplot(dataset['SalePrice'], whis=10, color="#00B8D9");

This shows us the minimum and maximum values of `SalePrice`. It shows us also the three quartiles represented by the box and the vertical line inside of it. Lastly, we plot the histogram of the variable to see a more detailed view of the distribution:

In [ ]:
sns.distplot(dataset['SalePrice'], kde=False, 
             color="#172B4D", hist_kws={"alpha": 0.8});
plt.ylabel("Count");

## Correlation Between Variables



In [ ]:
fig, ax = plt.subplots(figsize=(12,9))
sns.heatmap(dataset.corr(), ax=ax);

In [ ]:
sns.distplot(dataset['SalePrice'], kde=False, 
             color="#172B4D", hist_kws={"alpha": 0.8});
plt.ylabel("Count");

We can see that most house prices fall between 100,000 and 200,000. We see also that there is a number of expensive houses to the right of the plot. Now, we move to see the distribution of `Overall Qual` variable:

In [ ]:
sns.distplot(dataset['Overall Qual'], kde=False, 
             color="#172B4D", hist_kws={"alpha": 1});
plt.ylabel("Count");

We see that `Overall Qual` takes an integer value between 1 and 10, and that most houses have an overall quality between 5 and 7. Now we plot the scatter plot of `SalePrice` and `Overall Qual` to see the relationship between them:

In [ ]:
plt.scatter(x=dataset['Overall Qual'], y=dataset['SalePrice'], 
            color="orange", edgecolors="#000000", linewidths=0.5);
plt.xlabel("Overall Qual"); plt.ylabel("SalePrice");

We can see that they are truly positively correlated; generally, as the overall quality increases, the sale price increases too. This verfies what we got from the heatmap above.

Now, we want to see the relationship between the target variable and `Gr Liv Area` variable which represents the living area above ground. Let us first see the distribution of `Gr Liv Area`:

In [ ]:
sns.distplot(dataset['Gr Liv Area'], kde=False, 
             color="#172B4D", hist_kws={"alpha": 0.8});
plt.ylabel("Count");

We can see that the above-ground living area falls approximately between 800 and 1800 ft<sup>2</sup>. Now, let us see the relationship between `Gr Liv Area` and the target variable:

In [ ]:
plt.scatter(x=dataset['Gr Liv Area'], y=dataset['SalePrice'], 
            color="orange", edgecolors="#000000", linewidths=0.5);
plt.xlabel("Gr Liv Area"); plt.ylabel("SalePrice");

The scatter plot above shows clearly the strong positive correlation between `Gr Liv Area` and `SalePrice` verifying what we found with the heatmap.

#### Moderate Positive Correlation

Next, we want to visualize the relationship between the target variable and the variables that are positively correlated with it, but the correlation is not very strong. Namely, these variables are `Year Built`, `Year Remod/Add`, `Mas Vnr Area`, `Total Bsmt SF`, `1st Flr SF`, `Full Bath`, `Garage Cars`, and `Garage Area`. We start with the first four. Let us see the distribution of each of them:

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18,5))
fig.subplots_adjust(hspace=0.5, wspace=0.6)
for ax, v in zip(axes.flat, ["Year Built", "Year Remod/Add", 
                             "Mas Vnr Area", "Total Bsmt SF"]):
    sns.distplot(dataset[v], kde=False, color="#172B4D", 
                 hist_kws={"alpha": 0.8}, ax=ax)
    ax.set(ylabel="Count");

Now let us see their relationships with the target variable using scatter plots:

In [ ]:
x_vars = ["Year Built", "Year Remod/Add", "Mas Vnr Area", "Total Bsmt SF"]
g = sns.PairGrid(dataset, y_vars=["SalePrice"], x_vars=x_vars);
g.map(plt.scatter, color="orange", edgecolors="#000000", linewidths=0.5);

Next, we move to the last four. Let us see the distribution of each of them:

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18,5))
fig.subplots_adjust(hspace=0.5, wspace=0.6)
for ax, v in zip(axes.flat, ["1st Flr SF", "Full Bath", 
                             "Garage Cars", "Garage Area"]):
    sns.distplot(dataset[v], kde=False, color="#172B4D", 
                 hist_kws={"alpha": 0.8}, ax=ax);
    ax.set(ylabel="Count");

And now let us see their relationships with the target variable:

In [ ]:
x_vars = ["1st Flr SF", "Full Bath", "Garage Cars", "Garage Area"]
g = sns.PairGrid(dataset, y_vars=["SalePrice"], x_vars=x_vars);
g.map(plt.scatter, color="orange", edgecolors="#000000", linewidths=0.5);

From the plots above, we can see that these eight variables are truly positively correlated with the target variable. However, it's apparent that they are not as highly correlated as `Overall Qual` and `Gr Liv Area`.

### Relatioships Between Predictor Variables

#### Positive Correlation

Apart from the target variable, when we plotted the heatmap, we discovered a high positive correlation between `Garage Cars` and `Garage Area` and between `Gr Liv Area` and `TotRms AbvGrd`. We want to visualize these correlations also. We've already seen the distribution of each of them except for `TotRms AbvGrd`. Let us see the distribution of `TotRms AbvGrd` first:

In [ ]:
sns.distplot(dataset['TotRms AbvGrd'], kde=False, 
             color="#172B4D", hist_kws={"alpha": 0.8});
plt.ylabel("Count");

Now, we visualize the relationship between `Garage Cars` and `Garage Area` and between `Gr Liv Area` and `TotRms AbvGrd`:

In [ ]:
plt.rc("grid", linewidth=0.05)
fig, axes = plt.subplots(1, 2, figsize=(15,5))
fig.subplots_adjust(hspace=0.5, wspace=0.4)
h1 = axes[0].hist2d(dataset["Garage Cars"], 
                    dataset["Garage Area"],
                    cmap="viridis");
axes[0].set(xlabel="Garage Cars", ylabel="Garage Area")
plt.colorbar(h1[3], ax=axes[0]);
h2 = axes[1].hist2d(dataset["Gr Liv Area"], 
                    dataset["TotRms AbvGrd"],
                    cmap="viridis");
axes[1].set(xlabel="Gr Liv Area", ylabel="TotRms AbvGrd")
plt.colorbar(h1[3], ax=axes[1]);
plt.rc("grid", linewidth=0.25)

We can see the strong correlation between each pair. For `Garage Cars` and `Garage Area`, we see that the highest concentration of data is when `Garage Cars` is 2 and `Garage Area` is approximately between 450 and 600 ft<sup>2</sup>. For `Gr Liv Area` and `TotRms AbvGrd`, we notice that the highest concentration is when `Garage Liv Area` is roughly between 800 and 2000 ft<sup>2</sup> and `TotRms AbvGrd` is 6.

#### Negative Correlation

When we plotted the heatmap, we also discovered a significant negative correlation between `Bsmt Unf SF` and `BsmtFin SF 1`, and between `Bsmt Unf SF` and `Bsmt Full Bath`. We also want to visualize these correlations. Let us see the distribution of these variables first:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16,5))
fig.subplots_adjust(hspace=0.5, wspace=0.6)
for ax, v in zip(axes.flat, ["Bsmt Unf SF", "BsmtFin SF 1", "Bsmt Full Bath"]):
    sns.distplot(dataset[v], kde=False, color="#172B4D", 
                 hist_kws={"alpha": 0.8}, ax=ax);
    ax.set(ylabel="Count")

Now, we visualize the relationship between each pair using scatter plots:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
fig.subplots_adjust(hspace=0.5, wspace=0.4)
axes[0].scatter(dataset["Bsmt Unf SF"], dataset["BsmtFin SF 1"],
                color="orange", edgecolors="#000000", linewidths=0.5);
axes[0].set(xlabel="Bsmt Unf SF", ylabel="BsmtFin SF 1");
axes[1].scatter(dataset["Bsmt Unf SF"], dataset["Bsmt Full Bath"],
                color="orange", edgecolors="#000000", linewidths=0.5);
axes[1].set(xlabel="Bsmt Unf SF", ylabel="Bsmt Full Bath");

In [ ]:
for f in ["Overall Qual", "Gr Liv Area"]:
    dataset[f + "_p2"] = dataset[f] ** 2
    dataset[f + "_p3"] = dataset[f] ** 3
dataset["OverallQual_GrLivArea"] = \
    dataset["Overall Qual"] * dataset["Gr Liv Area"]

In [ ]:
dataset.drop(["Garage Cars", "TotRms AbvGrd"], axis=1, inplace=True)

### Dealing with Ordinal Variables

There are some ordinal features in our dataset. For example, the `Bsmt Cond` feature has the following possible values: 

In [ ]:
print("Unique values in 'Bsmt Cond' column:")
print(dataset['Bsmt Cond'].unique().tolist())

In [ ]:
mp = {'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0}
dataset['Exter Qual'] = dataset['Exter Qual'].map(mp)
dataset['Exter Cond'] = dataset['Exter Cond'].map(mp)
dataset['Heating QC'] = dataset['Heating QC'].map(mp)
dataset['Kitchen Qual'] = dataset['Kitchen Qual'].map(mp)

mp = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Basement':0}
dataset['Bsmt Qual'] = dataset['Bsmt Qual'].map(mp)
dataset['Bsmt Cond'] = dataset['Bsmt Cond'].map(mp)
dataset['Bsmt Exposure'] = dataset['Bsmt Exposure'].map(
    {'Gd':4,'Av':3,'Mn':2,'No':1,'No Basement':0})

mp = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'No Basement':0}
dataset['BsmtFin Type 1'] = dataset['BsmtFin Type 1'].map(mp)
dataset['BsmtFin Type 2'] = dataset['BsmtFin Type 2'].map(mp)

dataset['Central Air'] = dataset['Central Air'].map({'Y':1,'N':0})
dataset['Functional'] = dataset['Functional'].map(
    {'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,
     'Maj2':2,'Sev':1,'Sal':0})
dataset['Fireplace Qu'] = dataset['Fireplace Qu'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Fireplace':0})
dataset['Garage Finish'] = dataset['Garage Finish'].map(
    {'Fin':3,'RFn':2,'Unf':1,'No Garage':0})
dataset['Garage Qual'] = dataset['Garage Qual'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0})
dataset['Garage Cond'] = dataset['Garage Cond'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0})
dataset['Pool QC'] = dataset['Pool QC'].map(
    {'Ex':4,'Gd':3,'TA':2,'Fa':1,'No Pool':0})
dataset['Land Slope'] = dataset['Land Slope'].map(
    {'Sev': 2, 'Mod': 1, 'Gtl': 0})
dataset['Fence'] = dataset['Fence'].map(
    {'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'No Fence':0})

In [ ]:
dataset[['Paved Drive']].head()

Now, we perform one-hot encoding:

In [ ]:
dataset = pd.get_dummies(dataset)

Let us see what has happened to the `Paved Drive` variable by looking at the same rows above: 

In [ ]:
pavedDrive_oneHot = [c for c in dataset.columns if c.startswith("Paved")]
dataset[pavedDrive_oneHot].head()

In [ ]:
dataset[['SalePrice']].head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# We need to fit the scaler to our data before transformation
dataset.loc[:, dataset.columns != 'SalePrice'] = scaler.fit_transform(
    dataset.loc[:, dataset.columns != 'SalePrice'])

## Splitting the Dataset

As usual for supervised machine learning problems, we need a training dataset to train our model and a test dataset to evaluate the model. So we will split our dataset randomly into two parts, one for training and the other for testing. For that, we will use another function from Scikit-Learn called `train_test_split()`:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    dataset.drop('SalePrice', axis=1), dataset[['SalePrice']], 
    test_size=0.25, random_state=3)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

parameter_space = {
    "alpha": [1, 10, 100, 290, 500],
    "fit_intercept": [True, False],
    "solver": ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
}

clf = GridSearchCV(Ridge(random_state=3), parameter_space, n_jobs=4,
                   cv=3, scoring="neg_mean_absolute_error")

clf.fit(X_train, y_train)
print("Best parameters:")
print(clf.best_params_)

We defined the parameter space above using reasonable values for chosen parameters. Then we used `GridSearchCV()` with 3 folds (`cv=3`). Now we build our Ridge model with the best parameters found:

In [ ]:
ridge_model = Ridge(random_state=3, **clf.best_params_)

Then we train our model using our training set (`X_train` and `y_train`):

In [ ]:
ridge_model.fit(X_train, y_train);

Finally, we test our model on `X_test`. Then we evaluate the model performance by comparing its predictions with the actual true values in `y_test` using the MAE metric as we described above:

In [ ]:
from sklearn.metrics import mean_absolute_error

y_pred = ridge_model.predict(X_test)
ridge_mae = mean_absolute_error(y_test, y_pred)
print("Ridge MAE =", ridge_mae)

#### 2. Elastic Net

This model has the following syntax:

```py
ElasticNet(alpha=1.0, l1_ratio=0.5, fit_intercept=True, normalize=False, 
           precompute=False, max_iter=1000, copy_X=True, tol=0.0001, 
           warm_start=False, positive=False, random_state=None, selection=’cyclic’)
```

Firstly, we will use `GridSearchCV()` to search for the best model parameters in a parameter space provided by us. The parameter `alpha` is a constant that multiplies the penalty terms, `l1_ratio` determines the amount of L1 and L2 regularizations, `fit_intercept` is the same as Ridge's.

In [ ]:
from sklearn.linear_model import ElasticNet

parameter_space = {
    "alpha": [1, 10, 100, 280, 500],
    "l1_ratio": [0.5, 1],
    "fit_intercept": [True, False],
}

clf = GridSearchCV(ElasticNet(random_state=3), parameter_space, 
                   n_jobs=4, cv=3, scoring="neg_mean_absolute_error")

clf.fit(X_train, y_train)
print("Best parameters:")
print(clf.best_params_)

We defined the parameter space above using reasonable values for chosen parameters. Then we used `GridSearchCV()` with 3 folds (`cv=3`). Now we build our Ridge model with the best parameters found:

In [ ]:
elasticNet_model = ElasticNet(random_state=3, **clf.best_params_)

Then we train our model using our training set (`X_train` and `y_train`):

In [ ]:
elasticNet_model.fit(X_train, y_train);

Finally, we test our model on `X_test`. Then we evaluate the model performance by comparing its predictions with the actual true values in `y_test` using the MAE metric as we described above:

In [ ]:
y_pred = elasticNet_model.predict(X_test)
elasticNet_mae = mean_absolute_error(y_test, y_pred)
print("Elastic Net MAE =", elasticNet_mae)

### Nearest Neighbors

For Nearest Neighbors, we will use an implementation of the k-nearest neighbors (KNN) algorithm provided by Scikit-Learn package.

The KNN model has the following syntax:

```py
KNeighborsRegressor(n_neighbors=5, weights=’uniform’, algorithm=’auto’, 
                    leaf_size=30, p=2, metric=’minkowski’, metric_params=None, 
                    n_jobs=None, **kwargs)
```

Firstly, we will use `GridSearchCV()` to search for the best model parameters in a parameter space provided by us. The parameter `n_neighbors` represents `k` which is the number of neighbors to use, `weights` determines the weight function used in prediction: `uniform` or `distance`, `algorithm` specifies the algorithm used to compute the nearest neighbors, `leaf_size` is passed to `BallTree` or `KDTree` algorithm. It can affect the speed of the construction and query, as well as the memory required to store the tree.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

parameter_space = {
    "n_neighbors": [9, 10, 11,50],
    "weights": ["uniform", "distance"],
    "algorithm": ["ball_tree", "kd_tree", "brute"],
    "leaf_size": [1,2,20,50,200]
}

clf = GridSearchCV(KNeighborsRegressor(), parameter_space, cv=3, 
                   scoring="neg_mean_absolute_error", n_jobs=4)

clf.fit(X_train, y_train)
print("Best parameters:")
print(clf.best_params_)

We defined the parameter space above using reasonable values for chosen parameters. Then we used `GridSearchCV()` with 3 folds (`cv=3`). Now we build our Ridge model with the best parameters found:

In [ ]:
knn_model = KNeighborsRegressor(**clf.best_params_)

Then we train our model using our training set (`X_train` and `y_train`):

In [ ]:
knn_model.fit(X_train, y_train);

Finally, we test our model on `X_test`. Then we evaluate the model performance by comparing its predictions with the actual true values in `y_test` using the MAE metric as we described above:

In [ ]:
y_pred = knn_model.predict(X_test)
knn_mae = mean_absolute_error(y_test, y_pred)
print("K-Nearest Neighbors MAE =", knn_mae)

### Support Vector Regression

For Support Vector Regression (SVR), we will use one of three implementations provided by the Scikit-Learn package.

The SVR model has the following syntax:

```py
SVR(kernel=’rbf’, degree=3, gamma=’auto_deprecated’, coef0=0.0, tol=0.001, 
    C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
```

Firstly, we will use `GridSearchCV()` to search for the best model parameters in a parameter space provided by us. The parameter `kernel` specifies the kernel type to be used in the algorithm, `degree` represents the degree of the polynomial kernel `poly`, `gamma` is the kernel coefficient for `rbf`, `poly` and `sigmoid` kernels, `coef0` is independent term in kernel function, and `C` is the penalty parameter of the error term.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR

parameter_space = \
    {
        "kernel": ["poly", "linear", "rbf", "sigmoid"],
        "degree": [3, 5],
        "coef0": [0, 3, 7],
        "gamma":[1e-3, 1e-1, 1/X_train.shape[1]],
        "C": [1, 10, 100],
    }

clf = GridSearchCV(SVR(), parameter_space, cv=3, n_jobs=4,
                   scoring="neg_mean_absolute_error")

clf.fit(X_train, y_train)
print("Best parameters:")
print(clf.best_params_)

We defined the parameter space above using reasonable values for chosen parameters. Then we used `GridSearchCV()` with 3 folds (`cv=3`). Now we build our Support Vector Regression model with the best parameters found:

In [ ]:
svr_model = SVR(**clf.best_params_)

Then we train our model using our training set (`X_train` and `y_train`):

In [ ]:
svr_model.fit(X_train, y_train);

Finally, we test our model on `X_test`. Then we evaluate the model performance by comparing its predictions with the actual true values in `y_test` using the MAE metric as we described above:

In [ ]:
y_pred = svr_model.predict(X_test)
svr_mae = mean_absolute_error(y_test, y_pred)
print("Support Vector Regression MAE =", svr_mae)

### Decision Tree

For Decision Tree (DT), we will use an implementations provided by the Scikit-Learn package.

The Decision Tree model has the following syntax:

```py
DecisionTreeRegressor(criterion=’mse’, splitter=’best’, max_depth=None, 
                      min_samples_split=2, min_samples_leaf=1, 
                      min_weight_fraction_leaf=0.0, max_features=None, 
                      random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                      min_impurity_split=None, presort=False)
```

Firstly, we will use `GridSearchCV()` to search for the best model parameters in a parameter space provided by us. The parameter `criterion` specifies the function used to measure the quality of a split, `min_samples_split` determines the minimum number of samples required to split an internal node, `min_samples_leaf` determines the minimum number of samples required to be at a leaf node, and `max_features` controls the number of features to consider when looking for the best split.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

parameter_space = \
    {
        "criterion": ["mse", "friedman_mse", "mae"],
        "min_samples_split": [5, 18, 29, 50],
        "min_samples_leaf": [3, 7, 15, 25],
        "max_features": [20, 50, 150, 200, X_train.shape[1]],
    }

clf = GridSearchCV(DecisionTreeRegressor(random_state=3), parameter_space, 
                   cv=3, scoring="neg_mean_absolute_error", n_jobs=4)

clf.fit(X_train, y_train)
print("Best parameters:")
print(clf.best_params_)

We defined the parameter space above using reasonable values for chosen parameters. Then we used `GridSearchCV()` with 3 folds (`cv=3`). Now we build our Decision Tree model with the best parameters found:

In [ ]:
dt_model = DecisionTreeRegressor(**clf.best_params_)

Then we train our model using our training set (`X_train` and `y_train`):

In [ ]:
dt_model.fit(X_train, y_train);

Finally, we test our model on `X_test`. Then we evaluate the model performance by comparing its predictions with the actual true values in `y_test` using the MAE metric as we described above:

In [ ]:
y_pred = dt_model.predict(X_test)
dt_mae = mean_absolute_error(y_test, y_pred)
print("Decision Tree MAE =", dt_mae)

### Neural Network

For Neural Network (NN), we will use an implementations provided by the Scikit-Learn package.

The Neural Network model has the following syntax:

```py
MLPRegressor(hidden_layer_sizes=(100, ), activation=’relu’, solver=’adam’, 
             alpha=0.0001, batch_size=’auto’, learning_rate=’constant’, 
             learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
             random_state=None, tol=0.0001, verbose=False, warm_start=False, 
             momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
             validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
             n_iter_no_change=10)
```

Firstly, we will use `GridSearchCV()` to search for the best model parameters in a parameter space provided by us. The parameter `hidden_layer_sizes` is a list where its ith element represents the number of neurons in the ith hidden layer, `activation` specifies the activation function for the hidden layer, `solver` determines the solver for weight optimization, and `alpha` represents L2 regularization penalty.

In [ ]:
from sklearn.neural_network import MLPRegressor

parameter_space = \
    {
        "hidden_layer_sizes": [(7,)*3, (19,), (100,), (154,)],
        "activation": ["identity", "logistic", "tanh", "relu"],
        "solver": ["lbfgs"],
        "alpha": [1, 10, 100],
    }

clf = GridSearchCV(MLPRegressor(random_state=3), parameter_space, 
                   cv=3, scoring="neg_mean_absolute_error", n_jobs=4)

clf.fit(X_train, y_train)
print("Best parameters:")
print(clf.best_params_)

We defined the parameter space above using reasonable values for chosen parameters. Then we used `GridSearchCV()` with 3 folds (`cv=3`). Now we build our Neural Network model with the best parameters found:

In [ ]:
nn_model = MLPRegressor(**clf.best_params_)

Then we train our model using our training set (`X_train` and `y_train`):

In [ ]:
nn_model.fit(X_train, y_train);

Finally, we test our model on `X_test`. Then we evaluate the model performance by comparing its predictions with the actual true values in `y_test` using the MAE metric as we described above:

In [ ]:
y_pred = nn_model.predict(X_test)
nn_mae = mean_absolute_error(y_test, y_pred)
print("Neural Network MAE =", nn_mae)

### Random Forest

For Random Forest (RF), we will use an implementations provided by the Scikit-Learn package.

The Random Forest model has the following syntax:

```py
RandomForestRegressor(n_estimators=’warn’, criterion=’mse’, max_depth=None, 
                      min_samples_split=2, min_samples_leaf=1, 
                      min_weight_fraction_leaf=0.0, max_features=’auto’, 
                      max_leaf_nodes=None, min_impurity_decrease=0.0, 
                      min_impurity_split=None, bootstrap=True, oob_score=False, 
                      n_jobs=None, random_state=None, verbose=0, warm_start=False)
```

Firstly, we will use `GridSearchCV()` to search for the best model parameters in a parameter space provided by us. The parameter `n_estimators` specifies the number of trees in the forest, `bootstrap` determines whether bootstrap samples are used when building trees. `criterion`, `max_depth`, `min_samples_split`, `min_samples_leaf`, `max_features` are the same as those of the decision tree model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

parameter_space = \
    {
        "n_estimators": [10, 100, 300, 600],
        "criterion": ["mse", "mae"],
        "max_depth": [7, 50, 254],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 5],
        "max_features": [19, 100, X_train.shape[1]],
        "bootstrap": [True, False],
    }

clf = RandomizedSearchCV(RandomForestRegressor(random_state=3), 
                         parameter_space, cv=3, n_jobs=4,
                         scoring="neg_mean_absolute_error", 
                         n_iter=10, random_state=3)

clf.fit(X_train, y_train)
print("Best parameters:")
print(clf.best_params_)

We defined the parameter space above using reasonable values for chosen parameters. Then we used `GridSearchCV()` with 3 folds (`cv=3`). Now we build our Random Forest model with the best parameters found:

In [ ]:
rf_model = RandomForestRegressor(**clf.best_params_)

Then we train our model using our training set (`X_train` and `y_train`):

In [ ]:
rf_model.fit(X_train, y_train);

Finally, we test our model on `X_test`. Then we evaluate the model performance by comparing its predictions with the actual true values in `y_test` using the MAE metric as we described above:

In [ ]:
y_pred = rf_model.predict(X_test)
rf_mae = mean_absolute_error(y_test, y_pred)
print("Random Forest MAE =", rf_mae)

### Gradient Boosting

For Gradient Boosting (GB), we will use the renowned XGBoost implementations.

XGBoost model has the following syntax:

```py
XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, 
             objective='reg:linear', booster='gbtree', n_jobs=1, nthread=None, 
             gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, 
             colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, 
             scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, 
             missing=None, importance_type='gain', **kwargs)
```

Firstly, we will use `GridSearchCV()` to search for the best model parameters in a parameter space provided by us. The parameter `max_depth` sets the maximum depth of a tree, `learning_rate` represents the step size shrinkage used in updating weights, `n_estimators` specifies the number of boosted trees to fit, `booster` determines which booster to use, `gamma` specifies the minimum loss reduction required to make a further partition on a leaf node of the tree, `subsample` is subsample ratio of the training instances; this subsampling will occur once in every boosting iteration, `colsample_bytree` specifies the subsample ratio of columns when constructing each tree, `colsample_bylevel` specifies the subsample ratio of columns for each split, in each level, `reg_alpha` is L1 regularization term, and `reg_lambda` is L2 regularization term.

In [ ]:
from xgboost import XGBRegressor

parameter_space = \
    {
        "max_depth": [4, 5, 6],
        "learning_rate": [0.005, 0.009, 0.01],
        "n_estimators": [700, 1000, 2500],
        "booster": ["gbtree",],
        "gamma": [7, 25, 100],
        "subsample": [0.3, 0.6],
        "colsample_bytree": [0.5, 0.7],
        "colsample_bylevel": [0.5, 0.7,],
        "reg_alpha": [1, 10, 33],
        "reg_lambda": [1, 3, 10],
    }

clf = RandomizedSearchCV(XGBRegressor(random_state=3), 
                         parameter_space, cv=3, n_jobs=4,
                         scoring="neg_mean_absolute_error", 
                         random_state=3, n_iter=10)

clf.fit(X_train, y_train)
print("Best parameters:")
print(clf.best_params_)

We defined the parameter space above using reasonable values for chosen parameters. Then we used `GridSearchCV()` with 3 folds (`cv=3`). Now we build our XGBoost model with the best parameters found:

In [ ]:
xgb_model = XGBRegressor(**clf.best_params_)

Then we train our model using our training set (`X_train` and `y_train`):

In [ ]:
xgb_model.fit(X_train, y_train);

Finally, we test our model on `X_test`. Then we evaluate the model performance by comparing its predictions with the actual true values in `y_test` using the MAE metric as we described above:

In [ ]:
y_pred = xgb_model.predict(X_test)
xgb_mae = mean_absolute_error(y_test, y_pred)
print("XGBoost MAE =", xgb_mae)

In [ ]:
x = ['KNN', 'Decision Tree', 'Neural Network', 'Ridge', 
     'Elastic Net', 'Random Forest', 'SVR', 'XGBoost']
y = [22780.14, 20873.95, 15656.38, 15270.46, 14767.91,
     14506.46, 12874.93, 12556.68]
colors = ["#392834", "#5a3244", "#7e3c4d", "#a1484f", 
          "#c05949", "#d86f3d", "#e88b2b", "#edab06"]
fig, ax = plt.subplots()
plt.barh(y=range(len(x)), tick_label=x, width=y, height=0.4, color=colors);
ax.set(xlabel="MAE (smaller is better)", ylabel="Model");

In [ ]:
sns.violinplot(x=dataset['SalePrice'], inner="quartile", color="#36B37E");

In [ ]:
sns.boxplot(dataset['SalePrice'], whis=10, color="#00B8D9");

In [ ]:
sns.distplot(dataset['SalePrice'], kde=False,
             color="#172B4D", hist_kws={"alpha": 0.8});

From the three plots above, we can understand the distribution of `SalePrice`. Now let's get some numerical statistical information about it:

In [ ]:
y_train.describe(include=[np.number])

We can see that the mean is `179,846.69` and the median is `159,895`. We can see also that the first quartile is `128,500`; this means that 75% of the data is larger than this number. Now looking at XGBoost error of `12,556.68`, we can say that an error of about `12,000` is good for data whose mean is `159,895` and whose 75% of it is larger than `128,500`.

## Feature Importances

Some of the models we used provide the ability to see the importance of each feature in the dataset after fitting the model. We will look at the feature importances provided by both XGBoost and Random Forest models. We have 242 features in our data which is a big number, so we will take a look at the 15 most important features.

### XGBoost

Let's discover the most important features as determined by XGBoost model:

In [ ]:
xgb_feature_importances = xgb_model.feature_importances_
xgb_feature_importances = pd.Series(
    xgb_feature_importances, index=X_train.columns.values
    ).sort_values(ascending=False).head(15)

fig, ax = plt.subplots(figsize=(7, 5))
sns.barplot(x=xgb_feature_importances, 
            y=xgb_feature_importances.index, 
            color="#003f5c");
plt.xlabel('Feature Importance');
plt.ylabel('Feature');

### Random Forest

Now, let's see the most important features as for Random Forest model:

In [ ]:
rf_feature_importances = rf_model.feature_importances_
rf_feature_importances = pd.Series(
    rf_feature_importances, index=X_train.columns.values
    ).sort_values(ascending=False).head(15)

fig, ax = plt.subplots(figsize=(7,5))
sns.barplot(x=rf_feature_importances, 
            y=rf_feature_importances.index, 
            color="#ffa600");
plt.xlabel('Feature Importance');
plt.ylabel('Feature');

### Common Important Features

Now, let us see which features are among the most important features for both XGBoost and Random Forest models, and let's find out the difference in their importance regarding the two models:

In [ ]:
common_imp_feat = [x for x in xgb_feature_importances.index 
                   if x in rf_feature_importances.index]
commImpFeat_xgb_scores = [xgb_feature_importances[x] 
                          for x in common_imp_feat]
commImpFeat_rf_scores = [rf_feature_importances[x] 
                         for x in common_imp_feat]

ind = np.arange(len(commImpFeat_xgb_scores))
width = 0.35

fig, ax = plt.subplots()
ax.bar(ind - width/2, commImpFeat_xgb_scores, width,
       color='#003f5c', label='XGBoost');
ax.bar(ind + width/2, commImpFeat_rf_scores, width, 
       color='#ffa600', label='Random Forest')
ax.set_xticks(ind);
ax.set_xticklabels(common_imp_feat);
ax.legend();
plt.xticks(rotation=90);